# Imports et outils

In [1]:
import pandas as pd
import numpy as np

In [2]:
# Chaque repas est une somme d'ingrédients, ayant pour attributs un id et une quantité en grammes. Les ID correspondent à des lignes de la table des ingrédients.
class Ingredient():
  id: int
  gQuantity: float
  groupId: int = None

  def __init__(self, id, gQuantity, groupId=None):
      self.id = id
      self.gQuantity = gQuantity
      self.groupId = groupId

In [39]:
# Liste des nutriments pris en compte, de leur ID et de leur unité.
nut_dict = {'203': "Protéines", '204': "Lipides", '205': "Glucides", '208': "Energie", '291': "Fibres", '601': "Cholesterol", '255' : "Eau", '269': "Sucres", '810': "Amidon", '301': "Calcium", '304': "Magnesium", '305' : "Phosphore", '306': "Potassium", '307': "Sodium", '303': "Fer", '309' : "Zinc", '312': "Cuivre", '315': "Manganese", '317': "Selenium", '606': "AG Saturés", '645': "AG monoinsaturés", '646': "AG polyinsaturés", '617': "AG oléique", '618': "AG linoléique", '619': "AG alpha-linolénique", '620': "AG arachidonique", '629': "AG EPA", '621': "AG DHA", '319' : "Vitamine A (Retinol)", '321': "Vitamine A (B-carotene)", '339': "Vitamine D", '401': "Vitamine C", '404': "Vitamine B1", '405' : "Vitamine B2", '406': "Vitamine B3", '410': "Vitamine B5", '415': "Vitamine B6", '417': "Vitamine B9", '418': "Vitamine B12", '501' : "Tryptophane (AA)", '502': "Threonine (AA)", '503': "Isoleucine (AA)", '504': "Leucine (AA)", '505' : "Lysine (AA)", '506': "Methionine (AA)", '508': "Phenylalanine (AA)", '512' : "Histidine (AA)", '510' : "Valine (AA)", '3000': "Iode", '4000': "Vitamine E", '4001': "Vitamine K"}
unities_dict = {'203': "g/100 g", '204': "g/100 g", '205': "g/100 g", '208': "kcal/100 g", '291': "g/100 g", '601': "mg/100 g", '255' : "g/100 g", '269': "g/100 g", '810': "g/100 g", '301': "mg/100 g", '304': "mg/100 g", '305' : "mg/100 g", '306': "mg/100 g", '307': "mg/100 g", '303': "mg/100 g", '309' : "mg/100 g", '312': "mg/100 g", '315': "mg/100 g", '317': "µg/100 g", '606': "g/100 g",  '645': "g/100 g", '646': "g/100 g",  '617': "g/100 g", '618': "g/100 g",  '619': "g/100 g", '620': "g/100 g",  '629': "g/100 g", '621': "g/100 g", '319' : "µg/100 g", '321': "µg/100 g", '339': "µg/100 g", '401': "mg/100 g", '404': "mg/100 g", '405' : "mg/100 g", '406': "mg/100 g", '410': "mg/100 g", '415': "mg/100 g", '417': "µg/100 g", '418': "µg/100 g", '501' : "g/100 g", '502': "g/100 g", '503': "g/100 g", '504': "g/100 g", '505' : "g/100 g", '506': "g/100 g", '508': "g/100 g", '512' : "g/100 g", '510' : "g/100 g", '3000': "µg/100 g", '4000': "mg/100 g", '4001': "µg/100 g"} 

#  Fonctions annexes qui permettent de manipuler les données nutritionnelles
def build_DF_from_selection(Database: pd.DataFrame, foods: list[Ingredient]):
    """
    Cette méthode construit un Dataframe à partir d'une liste d'objets de type Ingredient. Elle permet de récupérer les informations nutritionnelles des aliments sélectionnés par l'utilisateur.
    
    Args:
        Database (pd.DataFrame): DataFrame contenant les informations nutritionnelles des aliments
        foods (list): Liste d'objets de type Ingredient
        
    Returns:
        selection_info_DF (pd.DataFrame): DataFrame contenant les informations nutritionnelles des aliments sélectionnés par l'utilisateur
    """

    food_ids = [obj.id for obj in foods]
    selection_info_DF = Database.loc[Database['FoodID'].isin(food_ids)].copy()

    for food in foods:  
        # Ajouter une colonne "portion" à partir de la liste d'objets
        quantity_list = []
        for i, row in selection_info_DF.iterrows():
            for food in foods:
                if row['FoodID'] == food.id:
                    quantity_list.append(food.gQuantity)
                    break
            else:
                quantity_list.append(None)

        selection_info_DF.loc[:, 'Quantity'] = quantity_list

        return selection_info_DF.set_index("FoodID", drop=False)

def sum_nutrients(selection: pd.DataFrame):
    """
    Cette méthode permet de sommer les nutriments des aliments sélectionnés par l'utilisateur, pour obtenir les valeurs nutritionnelles totales de la sélection.
    
    Args:
        selection (pd.DataFrame): DataFrame contenant les informations nutritionnelles des aliments sélectionnés par l'utilisateur
        
    Returns:
        total (pd.Series): Série contenant les valeurs nutritionnelles totales de la sélection
    """
    
    # Drop columns that are not nutrients
    columns_to_drop = ["FoodID", "FoodIDCNF", "FoodGroupID", "FoodName", "FoodGroupName", "OriginalFoodName", "EnglishFoodName", "FoodSubGroup", "FoodSubSubGroup", "AgribalyseAlternative", "ConversionFactorValue", "Quantity"]
    columns_present = [col for col in columns_to_drop if col in selection.columns]

    # Extract ratio and nutrients of the selection
    quantities = selection['Quantity']
    nutrients = selection.drop(columns_present, axis=1)

    # Total nutrient = sum of nutrient of each aliments, weighted by the ratio
    total = (nutrients.apply(lambda x: np.asarray(x) *
                np.asarray(quantities/100))).sum(numeric_only=True, axis=0)
    
    return total

def get_nutrients_from_meal(meal: list[Ingredient], ingredients_DB: pd.DataFrame):
    """
    Cette méthode utilise les 2 fonctions précédentes pour récupérer la somme des informations nutritionnelles des aliments sélectionnés par l'utilisateur.
    
    Args:
        meal (list): Liste d'objets de type Ingredient
        ingredients_DB (pd.DataFrame): DataFrame contenant les informations nutritionnelles des aliments
        
    Returns:
        nutrient_dict (dict): Dictionnaire contenant les informations nutritionnelles des aliments sélectionnés par l'utilisateur au format {nutrient_id: quantity}
    """
    
    # Get nutritional data for the selection
    selection_info_DF = build_DF_from_selection(
        ingredients_DB, meal)
    
    
    # Create a dict of nutrients for the selection
    summed_nutrient_info = sum_nutrients(selection_info_DF).to_dict()    
    
    # Prepare nutrient names
    nutrient_names = nut_dict

    # Prepare nutrient unities
    nutrient_unities = unities_dict
    for key, value in nutrient_unities.items():
        nutrient_unities[key] = value.split('/')[0]
        if nutrient_unities[key] =="Âµg":
            nutrient_unities[key] = "µg"

    # Prepare final dataframe
    name_df = pd.DataFrame(list(nutrient_names.items()), columns=['id', 'name'])
    quantities_df = pd.DataFrame(list(summed_nutrient_info.items()), columns=['id', 'quantity'])
    unities_df = pd.DataFrame(list(nutrient_unities.items()), columns=['id', 'unity'])

    nutrient_df = pd.merge(quantities_df, name_df, on='id')
    nutrient_df = pd.merge(nutrient_df,unities_df, on='id')
    
    nutrient_dict = dict(zip(nutrient_df["id"],nutrient_df["quantity"])) 

    return nutrient_dict

def sigmoid_piecewise(x, k1=5, k2=10, x0=0.4):
    """
    Sigmoïde avec deux pentes différentes selon que x est inférieur ou supérieur à x0.
    k1 : Pente pour x < x0
    k2 : Pente pour x >= x0
    x0 : Point central où y=0.5
    """
    y = np.where(
        x < x0,
        1 / (1 + np.exp(-k1 * (x - x0))),  # k1 pour x < x0
        1 / (1 + np.exp(-k2 * (x - x0)))   # k2 pour x >= x0
    )
    return y


# Données d'entrées

In [4]:
# Chargement de la base de données des ingrédients
ingredients_db = pd.read_csv("data/ingredients_db.csv", sep=';')

# Chargement de la base de données des recettes, pouvant être utilisée pour tester la cohérence des ingrédients ensemble
recipes_db = pd.read_csv('data/recipesDataset', index_col=0, converters={"ingredient_names": lambda x: x.strip("[]").replace("'", "").split(", ")})

# Chargement de la base de données des repas, pour suggérer des repas du soir
meals_db = pd.read_csv("data/meals.csv", sep=';')

In [35]:
userProfile = {"age": 35, "weight": 70, "size": 170, "activityLevel": 1.4}

userMeal1 = [
    Ingredient(9104, 150),
    Ingredient(20035, 80),
    Ingredient(11042, 1),
    Ingredient(20524, 130),
    Ingredient(11001, 3),
    Ingredient(11000, 2),
    Ingredient(6259, 125),
    Ingredient(20068, 20)
] # Chili con carne et Riz

userMeal2 = [
    Ingredient(23426, 50),
    Ingredient(28504, 50),
    Ingredient(19431, 47),
    Ingredient(22000, 45),
    Ingredient(19041, 50),
    Ingredient(11048, 1)
] # Quiche Lorraine

userMeal3 = [
    Ingredient(30155, 120),
    Ingredient(17270, 5),
    Ingredient(11112, 1),
    Ingredient(11001, 3),
    Ingredient(20047, 150),
    Ingredient(20009, 125),
    Ingredient(20020, 180),
    Ingredient(20532, 70),
    Ingredient(36006, 100),
    Ingredient(20068, 70),
    Ingredient(20033, 50)
] # Couscous

userMeal4 = [
    Ingredient(9811, 300),
    Ingredient(20260, 100),
    Ingredient(11000, 3),
    Ingredient(6255, 80),
    Ingredient(20034, 30)
] # Spaghetti bolognaise

userMeal5 = [
    Ingredient(20008, 60),
    Ingredient(20010, 70),
    Ingredient(20003, 40),
    Ingredient(4048, 100),
    Ingredient(20035, 30)
] # Bourguignon de légumes

userMeal6 = [
    Ingredient(7259, 80),
    Ingredient(36036, 130),
    Ingredient(25604, 20),
    Ingredient(20276, 30),
    Ingredient(12726, 30),
    Ingredient(11054, 20),
    Ingredient(20116, 30),
    Ingredient(20009, 30),
    Ingredient(4032, 100)
] # Burger de poulet & Frites

# Objectif 1 : Suggestions d'aliments

### Méthodes de calcul des Sous scores. A ne pas modifier, sauf pour récupérer plus de données si besoin.

In [40]:
# Calcul du sous-score énergétique, 
def compute_meal_energy_sub_score(summed_nutrients_info: dict, userProfile: dict):
    """
    Cette fonction calcule le score de l'énergie pour un repas, en fonction des besoins caloriques journaliers de l'utilisateur.\n
    *Inputs*:
        - summed_nutrients_info (Series): pandas Series contenant la somme des valeurs nutritionnelles du repas.
        - userProfile (dict): Profil de l'utilisateur contenant les clés "age", "weight", "size" et "activityLevel".
    *Outputs*:
        - energy_sub_score (float): score de l'énergie pour le repas (0 à 1).
    """

    meal_energy_in_kcal = summed_nutrients_info['208']

    # On utilise la formule de Black & al pour estimer les besoins caloriques journaliers
    daily_calory_needs = 1.083 * (userProfile["weight"] ** 0.48) * ((userProfile["size"]/100) ** 0.50) * (userProfile["age"] ** (-0.13)) * (1000 / 4.1855) * userProfile["activityLevel"]
    meal_calory_needs = daily_calory_needs / 3
    
    # Calcul de l'écart maximal (en fonction de x % de la valeur idéale)
    ecart_max = meal_calory_needs
    ecart_plage_max = meal_calory_needs * 0.2  # Plage avec score maximal

    # Calcul de l'écart absolu entre la valeur et la valeur idéale
    ecart = abs(meal_energy_in_kcal - meal_calory_needs)

    # Si l'écart dépasse la tolérance (tolerance_factor), le score est 0
    if ecart >= ecart_max:
        energy_sub_score = 0
    elif ecart <= ecart_plage_max:
        energy_sub_score = 1
    else:
        energy_sub_score = 1 - ((ecart - ecart_plage_max) / (ecart_max - ecart_plage_max))

    return energy_sub_score

# Calcul du sous-score macro nutritionnel
def compute_meal_macro_sub_score(current_meal: list[Ingredient], summed_nutrients_info: dict):
    """
    Calcule le score macro nutritionnel pour un repas donné en fonction des informations nutritionnelles résumées. 
    Le score est basé sur les proportions de légumes, protéines, glucides et lipides dans le repas, par rapport aux proportions optimales.\n

    Args:
        current_meal (list[schemas.Ingredient]): Liste des ingrédients du repas.
        summed_nutrients_info (pd.Series): Informations nutritionnelles résumées pour le repas.
        sum_g (float): Poids total du repas en grammes.

    Returns:
        float: Le score macro nutritionnel calculé.
    """
    
    # Calculer la quantité totale de légumes dans le repas
    sum_g = sum(ingredient.gQuantity for ingredient in current_meal)
    sum_vegetables_g = sum(ingredient.gQuantity for ingredient in current_meal if ingredient.groupId == 1)
    
    # Calculer la proportion de légumes dans le repas, limitée à 50%
    proportionVEGETABLES = min(sum_vegetables_g / sum_g, 0.5)

    # Estimation de l'énergie à partir des macronutriments
    estimated_energy_from_macronutrients = (
        summed_nutrients_info['203'] * 4 +  # Protéines
        summed_nutrients_info['205'] * 4 +  # Glucides
        summed_nutrients_info['204'] * 9    # Lipides
    )
    
    # Calculer les proportions des macronutriments par rapport à l'énergie estimée
    proportionPROT = summed_nutrients_info['203'] * 4 / estimated_energy_from_macronutrients
    proportionGLU = summed_nutrients_info['205'] * 4 / estimated_energy_from_macronutrients
    proportionLIP = summed_nutrients_info['204'] * 9 / estimated_energy_from_macronutrients

    # Proportions idéales des macronutriments
    proportion_idealVEGETABLES = 0.5
    proportion_idealePROT = 0.2
    proportion_idealeGLU = 0.45
    proportion_idealeLIP = 0.35

    # Calcul de la distance relative par rapport aux proportions idéales
    ecartVEGETABLES = abs(proportionVEGETABLES - proportion_idealVEGETABLES) / proportion_idealVEGETABLES
    ecartPROT = abs(proportionPROT - proportion_idealePROT) / proportion_idealePROT
    ecartGLU = abs(proportionGLU - proportion_idealeGLU) / proportion_idealeGLU
    ecartLIP = abs(proportionLIP - proportion_idealeLIP) / proportion_idealeLIP
    
    # Inverser les écarts pour obtenir un score où 0 est l'écart maximal et 1 est l'idéal parfait
    scoreVEGETABLES = max(0, 1 - ecartVEGETABLES)  # Score limité à 0 au minimum
    scorePROT = max(0, 1 - ecartPROT)  # Score limité à 0 au minimum
    scoreGLU = max(0, 1 - ecartGLU)  # Score limité à 0 au minimum
    scoreLIP = max(0, 1 - ecartLIP)  # Score limité à 0 au minimum
    
    # Moyenne des scores pour obtenir un score global
    score_macro = (scoreVEGETABLES + scorePROT + scoreGLU + scoreLIP) / 4
    
    return score_macro


### Score Général à optimiser

In [46]:

def compute_meal_score(current_meal: list[Ingredient], ingredients_DB: pd.DataFrame):
    """
    Calcule le score d'un repas en fonction des informations nutritionnelles résumées et du profil utilisateur.
    
    Args:
        summed_nutrients_info (pd.Series): Informations nutritionnelles résumées pour le repas.
        current_meal (list[schemas.Ingredient]): Liste des ingrédients du repas.
        user_profile (schemas.userProfile): Profil utilisateur.
    
    Returns:
        nut_score (float): Score nutritionnel du repas.
        energy_sub_score (float): Score de l'énergie du repas.
        macro_sub_score (float): Score macro nutritionnel du repas.
        nutri_sub_score (float): Score nutritionnel du rep
    """

    # Associer les groupes alimentaires aux ingrédients (nécessaire pour calculer la proportion de légumes)
    for ingredient in current_meal:
        ingredient.groupId = ingredients_db.loc[ingredients_db['FoodID'] == ingredient.id, 'FoodGroupID'].values[0]
    
    # Récupérer les informations nutritionnelles du repas
    summed_nutrients_info = get_nutrients_from_meal(current_meal, ingredients_DB)
    
    # Calculer les sous scores
    meal_energy_sub_score = compute_meal_energy_sub_score(summed_nutrients_info, userProfile=userProfile)
    meal_macro_sub_score = compute_meal_macro_sub_score(current_meal, summed_nutrients_info)
    
    # Calculer le score final
    meal_nut_sum = (1/3) * meal_energy_sub_score + (2/3) * meal_macro_sub_score
    meal_nut_score = sigmoid_piecewise(meal_nut_sum, k1=5, k2=7.5, x0=0.5)
    meal_nut_score = meal_nut_score.item()

    return meal_nut_score, meal_energy_sub_score, meal_macro_sub_score

In [47]:
compute_meal_score(userMeal6, ingredients_db)

(0.6735714193239601, 0.6812549219338006, 0.5542491089693035)

# Objectif 2 : Suggestions de plats

### Méthodes de calcul des Sous scores. A ne pas modifier, sauf pour récupérer plus de données si besoin.

In [48]:
# Calcul du sous-score énergétique, 
def compute_daily_energy_sub_score(summed_nutrients_info: dict, userProfile: dict):
    
    """
    Cette fonction calcule le score de l'énergie pour 2 repas, en fonction des besoins caloriques journaliers de l'utilisateur.\n
    *Inputs*:
        - summed_nutrients_info (Series): pandas Series contenant la somme des valeurs nutritionnelles du repas.
        - userProfile (dict): Profil de l'utilisateur contenant les clés "age", "weight", "size" et "activityLevel".
    *Outputs*:
        - energy_sub_score (float): score de l'énergie pour le repas (0 à 1).
    """

    meal_energy_in_kcal = summed_nutrients_info['208']

    # On utilise la formule de Black & al pour estimer les besoins caloriques journaliers
    daily_calory_needs = 1.083 * (userProfile["weight"] ** 0.48) * ((userProfile["size"]/100) ** 0.50) * (userProfile["age"] ** (-0.13)) * (1000 / 4.1855) * userProfile["activityLevel"]
    
    # On considère que les repas du soir et midi doivent représenter 80% des besoins journaliers
    daily_calory_needs = daily_calory_needs * 0.8 
    
    # Calcul de l'écart maximal (en fonction de x % de la valeur idéale)
    ecart_max = (daily_calory_needs / 2)
    ecart_plage_max = daily_calory_needs * 0.2  # Plage avec score maximal

    # Calcul de l'écart absolu entre la valeur et la valeur idéale
    ecart = abs(meal_energy_in_kcal - daily_calory_needs)

    # Si l'écart dépasse la tolérance (tolerance_factor), le score est 0
    if ecart >= ecart_max:
        energy_sub_score = 0
    elif ecart <= ecart_plage_max:
        energy_sub_score = 1
    else:
        energy_sub_score = 1 - ((ecart - ecart_plage_max) / (ecart_max - ecart_plage_max))

    return energy_sub_score

# Calcul du sous-score macro nutritionnel
def compute_daily_macro_sub_score(current_meal: list[Ingredient], summed_nutrients_info: dict):
    """
    Calcule le score macro nutritionnel pour 2 repas donnés en fonction des informations nutritionnelles résumées. 
    Le score est basé sur les proportions de légumes, et apports en protéines, glucides, lipides et fibres dans les repas, par rapport aux besoins journaliers.\n

    Args:
        current_meal (list[schemas.Ingredient]): Liste des ingrédients du repas.
        summed_nutrients_info (pd.Series): Informations nutritionnelles résumées pour le repas.

    Returns:
        float: Le score macro nutritionnel calculé.
    """
    
    # On utilise la formule de Black & al pour estimer les besoins caloriques journaliers
    daily_calory_needs = 1.083 * (userProfile["weight"] ** 0.48) * ((userProfile["size"]/100) ** 0.50) * (userProfile["age"] ** (-0.13)) * (1000 / 4.1855) * userProfile["activityLevel"]
    
    # On considère que les repas du soir et midi doivent représenter 80% des besoins journaliers
    daily_calory_needs = daily_calory_needs * 0.8 
    
    # Calculer les recommandations de macronutriments par rapport à l'énergie estimée
    daily_recommended_proteins_g = 0.15 * daily_calory_needs / 4
    daily_recommended_lipids_g = 0.38 * daily_calory_needs / 9
    daily_recommended_glucides_g = 0.47 * daily_calory_needs / 4
    daily_recommended_fibers_g = 12
    daily_recommended_vegetables_proportion = 0.4
    
    # Calculer la quantité totale de légumes dans le repas
    sum_g = sum(ingredient.gQuantity for ingredient in current_meal)
    sum_vegetables_g = sum(ingredient.gQuantity for ingredient in current_meal if ingredient.groupId == 1)
    
    # Calculer la proportion de légumes dans le repas, limitée à 50%
    proportionVEGETABLES = min(sum_vegetables_g / sum_g, 0.5)


    # Calcul de la distance relative par rapport aux proportions idéales
    ecartVEGETABLES = abs(proportionVEGETABLES - daily_recommended_vegetables_proportion) / daily_recommended_vegetables_proportion
    ecartFIBERS = abs(summed_nutrients_info['291'] - daily_recommended_fibers_g) / daily_recommended_fibers_g
    ecartPROT = abs (summed_nutrients_info['203'] - daily_recommended_proteins_g) / daily_recommended_proteins_g
    ecartGLU = abs(summed_nutrients_info['205'] - daily_recommended_glucides_g) / daily_recommended_glucides_g
    ecartLIP = abs(summed_nutrients_info['204'] - daily_recommended_lipids_g) / daily_recommended_lipids_g
    
    # Inverser les écarts pour obtenir un score où 0 est l'écart maximal et 1 est l'idéal parfait
    scoreVEGETABLES = max(0, 1 - ecartVEGETABLES)  # Score limité à 0 au minimum
    scorePROT = max(0, 1 - ecartPROT)  # Score limité à 0 au minimum
    scoreGLU = max(0, 1 - ecartGLU)  # Score limité à 0 au minimum
    scoreLIP = max(0, 1 - ecartLIP)  # Score limité à 0 au minimum
    scoreFIBERS = max(0, 1 - ecartFIBERS)  # Score limité à 0 au minimum
    
    # Moyenne des scores pour obtenir un score global
    score_macro = (scoreVEGETABLES + scorePROT + scoreGLU + scoreLIP + scoreFIBERS) / 5
    
    return score_macro


### Score Général à optimiser

In [49]:

def compute_daily_score(current_meal: list[Ingredient], second_meal: list[Ingredient], ingredients_DB: pd.DataFrame):
    """
    Calcule le score d'une journée en fonction des informations nutritionnelles résumées et du profil utilisateur.
    
    Args:
        summed_nutrients_info (pd.Series): Informations nutritionnelles résumées pour le repas.
        current_meal (list[schemas.Ingredient]): Liste des ingrédients du repas.
        user_profile (schemas.userProfile): Profil utilisateur.
    
    Returns:
        nut_score (float): Score nutritionnel du repas.
        energy_sub_score (float): Score de l'énergie du repas.
        macro_sub_score (float): Score macro nutritionnel du repas.
        nutri_sub_score (float): Score nutritionnel du rep
    """
    
    # Pour simplifier, on considère que consommer 2 repas est équivalent à consommer une fois la somme des 2 repas
    combined_meal = current_meal + second_meal
    
    # Associer les groupes alimentaires aux ingrédients (nécessaire pour calculer la proportion de légumes)
    for ingredient in combined_meal:
        ingredient.groupId = ingredients_db.loc[ingredients_db['FoodID'] == ingredient.id, 'FoodGroupID'].values[0]

    # Récupérer les informations nutritionnelles du repas
    summed_nutrients_info = get_nutrients_from_meal(combined_meal, ingredients_DB)
    
    daily_energy_sub_score = compute_meal_energy_sub_score(summed_nutrients_info, userProfile=userProfile)
    daily_macro_sub_score = compute_meal_macro_sub_score(combined_meal, summed_nutrients_info) 

    
    daily_nut_sum = (1/3) * daily_energy_sub_score + (2/3) * daily_macro_sub_score
    daily_nut_score = sigmoid_piecewise(daily_nut_sum, k1=5, k2=7.5, x0=0.5)
    daily_nut_score = daily_nut_score.item()

    return daily_nut_score, daily_energy_sub_score, daily_macro_sub_score


In [50]:
compute_daily_score(userMeal6, userMeal5, ingredients_db)

(0.7041696136535873, 0.4568025241221316, 0.6950453348666803)